In [24]:
#imports
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from pandas.tseries.offsets import BDay
import requests 

In [56]:
##top stocks 
stockPairList = [('AMZN','CRM'), ('MSFT', 'EBAY'), ('NVDA', 'TSM') ]
stockPairNames = [('Amazon', 'Salesforce'), ('Microsoft', 'Ebay'), ('NVIDIA', 'TSMC')]
url = 'https://newsapi.org/v2/everything?'

In [57]:
def get_trade_open(date):
    curr_date_open = pd.to_datetime(date).floor('d').replace(hour=13,minute=30) - BDay(0)
    curr_date_close = pd.to_datetime(date).floor('d').replace(hour=20,minute=0) - BDay(0)
    
    prev_date_close = (curr_date_open - BDay()).replace(hour=20,minute=0)
    next_date_open = (curr_date_close + BDay()).replace(hour=13,minute=30)
    
    if ((pd.to_datetime(date)>=prev_date_close) & (pd.to_datetime(date)<curr_date_open)):
        return curr_date_open
    elif ((pd.to_datetime(date)>=curr_date_close) & (pd.to_datetime(date)<next_date_open)):
        return next_date_open
    else:
        return None

In [58]:
def getStockSentiment(stock):
    parameters = {
        'q': stock, # query phrase
        'sortBy': 'popularity', # articles from popular sources and publishers come first
        'pageSize': 100,  # maximum is 100 for developer version
        'apiKey': '4a123522d48a4ed397f18a0b760dc4fc', # your own API key
    }

    response = requests.get(url, params=parameters)
    data = pd.DataFrame(response.json())

    news_df = pd.concat([data['articles'].apply(pd.Series)], axis=1)

    final_news = news_df.loc[:,['publishedAt','title']]
    final_news['publishedAt'] = pd.to_datetime(final_news['publishedAt'])
    final_news.sort_values(by='publishedAt',inplace=True)
    final_news["trading_time"] = final_news["publishedAt"].apply(get_trade_open)
    final_news = final_news[pd.notnull(final_news['trading_time'])]
    final_news['Date'] = pd.to_datetime(pd.to_datetime(final_news['trading_time']).dt.date)
    cs = []
    for row in range(len(final_news)):
        cs.append(analyzer.polarity_scores(final_news['title'].iloc[row])['compound'])

    final_news['compound_vader_score'] = cs
    final_news = final_news[(final_news[['compound_vader_score']] != 0).all(axis=1)].reset_index(drop=True)

    
    unique_dates = final_news['Date'].unique()
    grouped_dates = final_news.groupby(['Date'])
    keys_dates = list(grouped_dates.groups.keys())

    max_cs = []
    min_cs = []

    for key in grouped_dates.groups.keys():
        data = grouped_dates.get_group(key)
        if data["compound_vader_score"].max() > 0:
            max_cs.append(data["compound_vader_score"].max())
        elif data["compound_vader_score"].max() < 0:
            max_cs.append(0)

        if data["compound_vader_score"].min() < 0:
            min_cs.append(data["compound_vader_score"].min())
        elif data["compound_vader_score"].min() > 0:
            min_cs.append(0)

    extreme_scores_dict = {'Date':keys_dates,'max_scores':max_cs,'min_scores':min_cs}
    extreme_scores_df = pd.DataFrame(extreme_scores_dict)
    
    final_scores = []
    for i in range(len(extreme_scores_df)):
        final_scores.append(extreme_scores_df['max_scores'].values[i] + extreme_scores_df['min_scores'].values[i])

    extreme_scores_df['final_scores'] = final_scores

    return extreme_scores_df

In [60]:
returns = []
for pair in stockPairList:
    stock1 = pair[0]
    stock2 = pair[1]
    df = pd.read_csv('SentimentStocks/'+stock1+'_'+stock2+'.csv')
    df = df.drop('Unnamed: 0',1)
    df['Date']= pd.to_datetime(df['Date'])
    stock1Sentiment = getStockSentiment(stock1)
    stock2Sentiment = getStockSentiment(stock2)
    #merge df stock1Sentiment stock2Sentiment
    mergedSentimentDf = stock1Sentiment.merge(stock2Sentiment, how='inner', on='Date')
    mergedDf = df.merge(mergedSentimentDf, how = 'inner', on='Date')
    mergedDf = mergedDf.drop('returns',1)
    mergedDf['scores'] = mergedDf['final_scores_x'] + mergedDf['final_scores_y'] 
    for i in range (mergedDf.shape[0]):
            if mergedDf['z_score'].iloc[i] + mergedDf['scores'].iloc[i] < -1:
                mergedDf['position_1'].iloc[i] = 1
                mergedDf['position_2'].iloc[i] = -round(mergedDf['ratio'].iloc[i],0)
            if mergedDf['z_score'].iloc[i] + mergedDf['scores'].iloc[i] > 1:
                mergedDf['position_1'].iloc[i] = -1
                mergedDf['position_2'].iloc[i] = round(mergedDf['ratio'].iloc[i],0)
            if (abs(mergedDf['z_score'].iloc[i] + mergedDf['scores'].iloc[i]) < 1) & (abs(mergedDf['z_score'].iloc[i] + mergedDf['scores'].iloc[i]) > 0):
                mergedDf['position_1'].iloc[i] = 0
                mergedDf['position_2'].iloc[i] = 0
    mergedDf['returns'] = ((mergedDf[stock1]-mergedDf[stock1].shift(1))/mergedDf[stock1].shift(1))*mergedDf['position_1'].shift(1)+ ((mergedDf[stock2]-mergedDf[stock2].shift(1))/mergedDf[stock2].shift(1))*mergedDf['position_2'].shift(1)
    mergedDf.to_csv("SentimentStocks/Results/"+stock1+"_"+stock2+".csv")
    mergedDf['returns'].sum()
    returns.append(mergedDf['returns'].sum())
    
    

/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

In [61]:
returns

[-1.5481273470820176, 0.1218583390540198, -0.0003726051287071541]